CogVLM
====

**CogVLM: Visual Expert for Pretrained Language Models**

 * Paper: https://arxiv.org/pdf/2311.03079

![CogVLM Arch](../assets/cogvlm_arch.jpg)

## Installation

```bash
conda create -n cogvlm python=3.10 -y
conda activate cogvlm

python -m pip install --upgrade pip

# nvidia-smi => cuda 12.2 (12.1 works)
pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 \
  --extra-index-url https://download.pytorch.org/whl/cu121

pip install xformers==0.0.22 -f https://download.pytorch.org/whl/cu121/torch_stable.html

pip install -r requirements.txt

# download spacy data
python -m spacy download en_core_web_sm

python -c "import torch;print(torch.cuda.is_available(), torch.cuda.get_device_name(0))"

```

* Note:
Requires transformers 4.36.2 (newer versions won't work)
```bash
pip install --force-reinstall transformers==4.36.2
```

In [3]:
import argparse
from typing import Optional

import torch
from PIL import Image
from transformers import AutoModelForCausalLM, LlamaTokenizer

use_bf16 = False
use_fp16 = True
quant = True
device = "cuda" if torch.cuda.is_available() else "cpu"

# Choose appropriate floating point type
if use_bf16 and not use_fp16:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

tokenizer_name = "lmsys/vicuna-7b-v1.5"
model_name = "THUDM/cogvlm-chat-hf"

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name)

if quant:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        load_in_4bit=True,
        trust_remote_code=True,
    )
    model.eval();
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    model.to(device)
    model.eval();


Loading checkpoint shards:  38%|███▊      | 3/8 [00:04<00:06,  1.35s/it]